<a href="https://colab.research.google.com/github/Siva7/GoogleColabProject/blob/main/Kaggle/ImageClassification/IntelImageClassification/CustomNeuralNetDev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /root/.kaggle/
!cp "/content/drive/My Drive/Colab Notebooks/KaggleAPI/kaggle.json" /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d puneet6060/intel-image-classification

 98% 341M/346M [00:07<00:00, 43.0MB/s]
100% 346M/346M [00:07<00:00, 46.0MB/s]


In [6]:
import zipfile
with zipfile.ZipFile(r'/content/intel-image-classification.zip') as zfile:
  zfile.extractall(r'/content/intelImages')

#@title Default title text
#@ Image Size
All images are of size 150 * 150 

1) Use the raw data as is and analyse the predetions
2) Use data agumentation and see the difference.

In [7]:
import PIL
from PIL import Image
import numpy as np

image_sample = Image.open(r'/content/intelImages/seg_train/seg_train/buildings/0.jpg')
print(image_sample.mode)
print(image_sample.format)
print(image_sample.size)
img_arry  = np.array(image_sample)
print(img_arry.shape)
print(np.max(img_arry))

RGB
JPEG
(150, 150)
(150, 150, 3)
255


In [8]:
train_image_gen = ImageDataGenerator(rescale=1./255)

image_fimage_flow_genlow_gen = train_image_gen.flow_from_directory(r'/content/intelImages/seg_train/seg_train',
                                                     class_mode='categorical',
                                                     batch_size=32,
                                                     target_size=(150,150))

Found 14034 images belonging to 6 classes.


In [17]:

vgg_layer = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
avg_pool_layer = tfk.layers.GlobalAveragePooling2D()(vgg_layer.output)
final_dense_layer = tfk.layers.Dense(6,activation=tfk.activations.softmax)(avg_pool_layer)

model = tfk.models.Model(inputs=[vgg_layer.input],outputs=[final_dense_layer])
for layers in vgg_layer.layers:
  layers.trainable=False
model.summary()


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)  

In [20]:
model.compile(loss=tfk.losses.sparse_categorical_crossentropy,
              optimizer=tfk.optimizers.Adam(0.1),metrics=['acc'])

In [21]:
model.fit(image_fimage_flow_genlow_gen,epochs=2,steps_per_epoch=437)

Epoch 1/2


InvalidArgumentError: ignored